- *FEA.Dev*                   
    https://www.linkedin.com/company/fea-dev/  
    https://github.com/fea-dev-usp

### Autores:

- *Gustavo Yuji Yamachi*      
    https://www.linkedin.com/in/gustavo-yuji-osawa-yamachi-4193371a1/  
    https://github.com/Gust4242

- *Kevyn Andrade Marcelino*   
    https://www.linkedin.com/in/kevyn-a-marcelino/  
    https://github.com/k-marcelino


Data: 2023-08-26

Código auxiliar capaz de extrair dados da malha territorial de municipios diretamente da API do IBGE e gerar um arquivo .json com os dados a serem usados em mapas interativos

# -------------------------------------------------------------------------

In [ ]:
import json
import geopandas as gpd
import pandas as pd
import zipfile
from io import BytesIO
import requests

### Testando com 1 Município

In [ ]:
import requests

# Código do município desejado (exemplo: São Paulo, SP)
codigo_municipio = 3550308  # Código do IBGE para São Paulo/SP

# URL do serviço de mapas do IBGE para um município específico
ibge_url = f'https://servicodados.ibge.gov.br/api/v3/malhas/municipios/{codigo_municipio}?formato=application/vnd.geo+json&qualidade=maxima'

response = requests.get(ibge_url)

if response.status_code == 200:
    geojson_data = response.json()
    # Faça o que você quiser com os dados GeoJSON, como imprimir ou salvar em um arquivo
    print(geojson_data)
else:
    print("Falha ao obter os dados GeoJSON do município do IBGE")

#Iterar sobre os códigos de todos os municipios do Brasil e armazenar o GeoJSON de cada um único json


In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Lendo arquivo mais atual para pegar os códigos dos municípios

In [ ]:
pop = pd.read_excel('pop.xlsx')
pop

### Loop - Iterando sobre todos os códigos e juntando os arquivos json

In [ ]:
import requests
import json
from tqdm import tqdm


# Dicionário para armazenar os dados GeoJSON de cada município
geojson_collection = {
    "type": "FeatureCollection",
    "features": []
}

for codigo_municipio in tqdm(pop['CD_MUN']):
    ibge_url = f'https://servicodados.ibge.gov.br/api/v3/malhas/municipios/{codigo_municipio}?formato=application/vnd.geo+json&qualidade=maxima'
    
    response = requests.get(ibge_url, verify=False)
    
    if response.status_code == 200:
        geojson_data = response.json()
        geojson_collection["features"].append(geojson_data['features'][0])  # Append the feature part
    else:
        print(f"Falha ao obter os dados GeoJSON do município {codigo_municipio} do IBGE")

# Create a GeoJSON structure
merged_geojson = {
    "type": "FeatureCollection",
    "features": geojson_collection["features"]
}

# Save whole json collection
with open('geojson_2022.json', 'w') as output_file:
    json.dump(geojson_collection, output_file)

# Save the merged GeoJSON to a file
# with open('merged_geojson_2022.json', 'w') as output_file:
#     json.dump(merged_geojson, output_file)

print("GeoJSON completo foi salvo no arquivo 'geojson_2022.json'")
